# LearnDataEngineering Apache Iceberg Course - [Apache Spark Fundamentals in the Academy](https://learndataengineering.com/courses/1296953)

![learn-data-engineering](https://static-media.hotmart.com/NlvE9cql5MgqX2jBxM7BjiMUlL8=/0x60/https://uploads.teachablecdn.com/attachments/7hdFyMPwSMuYp7QIQg6u_lde_logo.png)

In [30]:
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [31]:
spark.sparkContext.setLogLevel("ERROR")  # or "WARN"
spark

## Explore the Iceberg catalog configuration in Apache Spark

In [32]:
%%sql
SHOW CATALOGS

catalog
default_cache_iceberg
demo
spark_catalog


In [33]:
!ls /home/iceberg/data_sync/

customer.parquet  orders.parquet    pokemon.csv
lineitem.parquet  part.parquet	    region.parquet
nation.parquet	  partsupp.parquet  supplier.parquet


## Create Database and Iceberg table with defined schema

In [34]:
%%sql
CREATE DATABASE IF NOT EXISTS academy

++
||
++
++

In [35]:
%%sql
SHOW DATABASES

namespace
academy


In [36]:
schema = StructType([
    StructField("number", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("type_1", StringType(), True),
    StructField("type_2", StringType(), True),
    StructField("abilities", StringType(), True),
    StructField("hp", IntegerType(), True),
    StructField("att", IntegerType(), True),
    StructField("def", IntegerType(), True),
    StructField("spa", IntegerType(), True),
    StructField("spd", IntegerType(), True),
    StructField("spe", IntegerType(), True),
    StructField("bst", IntegerType(), True),
    StructField("mean", DoubleType(), True),
    StructField("standard_deviation", DoubleType(), True),
    StructField("generation", DoubleType(), True),
    StructField("experience_type", StringType(), True),
    StructField("experience_to_level_100", IntegerType(), True),
    StructField("final_evolution", DoubleType(), True),
    StructField("catch_rate", IntegerType(), True),
    StructField("legendary", DoubleType(), True),
    StructField("mega_evolution", DoubleType(), True),
    StructField("alolan_form", DoubleType(), True),
    StructField("galarian_form", IntegerType(), True),
    StructField("against_normal", DoubleType(), True),
    StructField("against_fire", DoubleType(), True),
    StructField("against_water", DoubleType(), True),
    StructField("against_electric", DoubleType(), True),
    StructField("against_grass", DoubleType(), True),
    StructField("against_ice", DoubleType(), True),
    StructField("against_fighting", DoubleType(), True),
    StructField("against_poison", DoubleType(), True),
    StructField("against_ground", DoubleType(), True),
    StructField("against_flying", DoubleType(), True),
    StructField("against_psychic", DoubleType(), True),
    StructField("against_bug", DoubleType(), True),
    StructField("against_rock", DoubleType(), True),
    StructField("against_ghost", DoubleType(), True),
    StructField("against_dragon", DoubleType(), True),
    StructField("against_dark", DoubleType(), True),
    StructField("against_steel", DoubleType(), True),
    StructField("against_fairy", DoubleType(), True),
    StructField("height", DoubleType(), True),
    StructField("weight", DoubleType(), True),
    StructField("bmi", DoubleType(), True)
])

df = spark.read.schema(schema).csv("/home/iceberg/data_sync/pokemon.csv", header=True)

df.writeTo("academy.pokemon") \
  .using("iceberg") \
  .partitionedBy(f.col("type_1")) \
  .createOrReplace()

## Inspect Iceberg Metadata Tables
Metadata tables are virtual tables that Iceberg exposes so you can peek inside the catalog without touching the underlying file system.

### Snapshots
A snapshot is an immutable view of all data files and delete files that make up a table at a point‑in‑time.

In [37]:
%%sql
SELECT 
    *
FROM academy.pokemon.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-07-29 20:04:14.591000,288104134174738007,None,append,s3://warehouse/academy/pokemon/metadata/snap-288104134174738007-1-2397a9a0-35b6-4ed5-9049-d479d2ee05d1.avro,"{'engine-version': '3.5.5', 'added-data-files': '18', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'total-records': '1032', 'spark.app.id': 'local-1753816925020', 'changed-partition-count': '18', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '330021', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '330021', 'total-data-files': '18'}"
2025-07-29 20:04:38.727000,5675170136737628119,288104134174738007,overwrite,s3://warehouse/academy/pokemon/metadata/snap-5675170136737628119-1-0d802715-7de2-4395-b5a9-a0868ca30799.avro,"{'engine-version': '3.5.5', 'added-data-files': '18', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'deleted-data-files': '18', 'deleted-records': '1032', 'total-records': '1032', 'spark.app.id': 'local-1753816925020', 'removed-files-size': '330021', 'changed-partition-count': '18', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '215603', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '215603', 'total-data-files': '18'}"
2025-07-29 20:10:57.420000,6101732590538168055,5675170136737628119,replace,s3://warehouse/academy/pokemon/metadata/snap-6101732590538168055-1-f24a9592-0e98-46c8-a2d3-cf152b88e390.avro,"{'engine-version': '3.5.5', 'added-data-files': '134', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'deleted-data-files': '18', 'deleted-records': '1032', 'total-records': '1032', 'removed-files-size': '215603', 'changed-partition-count': '152', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '1180084', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '1180084', 'total-data-files': '134'}"
2025-07-29 20:12:43.539000,6600494381528787624,6101732590538168055,delete,s3://warehouse/academy/pokemon/metadata/snap-6600494381528787624-1-31c1e542-8492-4acf-897f-dc2433654bed.avro,"{'engine-version': '3.5.5', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'deleted-data-files': '17', 'deleted-records': '163', 'total-records': '869', 'spark.app.id': 'local-1753816925020', 'removed-files-size': '153072', 'changed-partition-count': '17', 'engine-name': 'spark', 'total-position-deletes': '0', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '1027012', 'total-data-files': '117'}"
2025-07-29 20:14:22.409000,4545212898047373911,None,append,s3://warehouse/academy/pokemon/metadata/snap-4545212898047373911-1-3206b9a9-9ccf-4310-beb8-71ce5eda9b12.avro,"{'engine-version': '3.5.5', 'added-data-files': '18', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'total-records': '1032', 'spark.app.id': 'local-1753816925020', 'changed-partition-count': '18', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '330021', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '330021', 'total-data-files': '18'}"


### Manifest files
Manifests are small Avro files that track groups of Parquet (or ORC) data files.

In [38]:
%%sql
SELECT *
FROM   academy.pokemon.manifests

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3://warehouse/academy/pokemon/metadata/3206b9a9-9ccf-4310-beb8-71ce5eda9b12-m0.avro,17064,0,4545212898047373911,18,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='Bug', upper_bound='Water')]"


### Files & Partitions

In [39]:
%%sql
SELECT * from academy.pokemon.partitions

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(type_1='Fire', generation=None)",0,64,1,18961,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Ghost', generation=None)",0,41,1,17382,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Bug', generation=None)",0,81,1,20057,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Electric', generation=None)",0,59,1,18486,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Ground', generation=None)",0,41,1,17418,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Fighting', generation=None)",0,42,1,17451,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Poison', generation=None)",0,40,1,17341,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Ice', generation=None)",0,38,1,17162,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Dragon', generation=None)",0,42,1,17323,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911
"Row(type_1='Dark', generation=None)",0,46,1,17665,0,0,0,0,2025-07-29 20:14:22.409000,4545212898047373911


In [40]:
%%sql
SELECT
    file_path,
    file_format,
    partition,
    record_count,
    value_counts,
    null_value_counts,
    nan_value_counts,
    lower_bounds,
    upper_bounds
FROM
  academy.pokemon.files

file_path,file_format,partition,record_count,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds
s3://warehouse/academy/pokemon/data/type_1=Steel/00000-162-2e97fa18-1ed9-4f32-8173-637259359ec6-0-00003.parquet,PARQUET,"Row(type_1='Steel', generation=None)",36,"{1: 36, 2: 36, 3: 36, 4: 36, 5: 36, 6: 36, 7: 36, 8: 36, 9: 36, 10: 36, 11: 36, 12: 36, 13: 36, 14: 36, 15: 36, 16: 36, 17: 36, 18: 36, 19: 36, 20: 36, 21: 36, 22: 36, 23: 36, 42: 36, 43: 36, 44: 36, 46: 36, 47: 36, 48: 36, 49: 36, 50: 36, 51: 36, 52: 36, 53: 36, 54: 36, 55: 36, 56: 36, 57: 36, 58: 36, 59: 36, 60: 36, 61: 36, 62: 36, 63: 36}","{1: 0, 2: 0, 3: 0, 4: 11, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 36, 42: 0, 43: 0, 44: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0}","{13: 0, 14: 0, 15: 0, 18: 0, 20: 0, 21: 0, 22: 0, 42: 0, 43: 0, 44: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0}","{1: bytearray(b'4\x00\x00\x00'), 2: bytearray(b'Aegislash Blade '), 3: bytearray(b'Steel'), 4: bytearray(b'Dragon'), 5: bytearray(b""[\'Battle Armor\',""), 6: bytearray(b'(\x00\x00\x00'), 7: bytearray(b'\x18\x00\x00\x00'), 8: bytearray(b'1\x00\x00\x00'), 9: bytearray(b'\x18\x00\x00\x00'), 10: bytearray(b'#\x00\x00\x00'), 11: bytearray(b'\x17\x00\x00\x00'), 12: bytearray(b'""\x01\x00\x00'), 13: bytearray(b'\xab\xaa\xaa\xaa\xaa*H@'), 14: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00\x00@'), 16: bytearray(b'Fast'), 17: bytearray(b'\x005\x0c\x00'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 19: bytearray(b'\x03\x00\x00\x00'), 20: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 21: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 22: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 42: bytearray(b'\x9a\x99\x99\x99\x99\x99\xc9?'), 43: bytearray(b'\x9a\x99\x99\x99\x99\x99\xf1?'), 44: bytearray(b'\xcd\xcc\xcc\xcc\xcc\xcc\x08@'), 46: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 47: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 48: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 49: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 50: bytearray(b'\x00\x00\x00\x00\x00\x00\xd0?'), 51: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 52: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 53: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 54: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 55: bytearray(b'\x00\x00\x00\x00\x00\x00\xd0?'), 56: bytearray(b'\x00\x00\x00\x00\x00\x00\xd0?'), 57: bytearray(b'\x00\x00\x00\x00\x00\x00\xd0?'), 58: bytearray(b'\x00\x00\x00\x00\x00\x00\xd0?'), 59: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 60: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 61: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 62: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 63: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?')}","{1: bytearray(b't\x03\x00\x00'), 2: bytearray(b'Steelix'), 3: bytearray(b'Steel'), 4: bytearray(b'Rock'), 5: bytearray(b""[\'Tough Claws\']""), 6: bytearray(b'\x87\x00\x00\x00'), 7: bytearray(b'\x91\x00\x00\x00'), 8: bytearray(b'\xe6\x00\x00\x00'), 9: bytearray(b'\x96\x00\x00\x00'), 10: bytearray(b'\x96\x00\x00\x00'), 11: bytearray(b'n\x00\x00\x00'), 12: bytearray(b'\xbc\x02\x00\x00'), 13: bytearray(b'\xac\xaa\xaa\xaa\xaa*]@'), 14: bytearray(b'\xeef\xe0\xd2\x0f*P@'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00 @'), 16: bytearray(b'Slow'), 17: bytearray(b'\xd0\x12\x13\x00'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 19: bytearray(b'\xff\x00\x00\x00'), 20: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 21: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 22: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 42: bytearray(b'\x00\x00\x00\x00\x00\x00%@'), 43: bytearray(b'33333?\x8f@'), 44: bytearray(b'\x00\x00\x00\x00\x00pw@'), 46: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 

## Analytical queries to get insights of our data

In [41]:
%%sql
SELECT
    type_1,
    MAX(bst)
FROM
    academy.pokemon
GROUP BY
    type_1
ORDER BY
  MAX(bst) DESC

type_1,max(bst)
Psychic,780
Dragon,780
Water,770
Ground,770
Fairy,720
Normal,720
Fighting,720
Steel,700
Rock,700
Poison,690


In [42]:
%%sql
SELECT
  p.*
FROM
  academy.pokemon p
  JOIN (
    SELECT
      type_1,
      MAX(bst) AS max_bst
    FROM
      academy.pokemon
    GROUP BY
      type_1
  ) max_bst_per_type ON p.type_1 = max_bst_per_type.type_1
  AND p.BST = max_bst_per_type.max_bst
ORDER BY
  p.type_1 DESC;

number,name,type_1,type_2,abilities,hp,att,def,spa,spd,spe,bst,mean,standard_deviation,generation,experience_type,experience_to_level_100,final_evolution,catch_rate,legendary,mega_evolution,alolan_form,galarian_form,against_normal,against_fire,against_water,against_electric,against_grass,against_ice,against_fighting,against_poison,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy,height,weight,bmi
382,Primal Kyogre,Water,None,['Primordial Sea'],100,150,90,180,160,90,770,128.33333333333334,36.24760528488591,6.0,Slow,1250000,1.0,3,1.0,1.0,0.0,None,1.0,0.5,0.5,2.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,9.8,430.0,4.5
376,Mega Metagross,Steel,Psychic,['Tough Claws'],80,145,150,105,110,110,700,116.66666666666669,24.09472049133493,6.0,Slow,1250000,1.0,3,0.0,1.0,0.0,None,0.5,2.0,1.0,1.0,0.5,0.5,1.0,0.0,2.0,0.5,0.25,1.0,0.5,2.0,0.5,2.0,0.5,0.5,2.5,942.9,150.9
248,Mega Tyranitar,Rock,Dark,['Sand Stream'],100,164,150,95,120,71,700,116.66666666666669,32.13340248955214,6.0,Slow,1250000,1.0,45,0.0,1.0,0.0,None,0.5,0.5,2.0,1.0,2.0,1.0,4.0,0.5,2.0,0.5,0.0,2.0,1.0,0.5,1.0,0.5,2.0,2.0,2.5,255.0,40.8
719,Mega Diancie,Rock,Fairy,['Magic Bounce'],50,160,110,160,110,110,700,116.66666666666669,37.26779962499649,6.0,Slow,1250000,1.0,3,1.0,1.0,0.0,None,0.5,0.5,2.0,1.0,2.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,1.0,0.0,0.5,4.0,1.0,1.1,27.8,23.0
150,Mega Mewtwo Y,Psychic,None,['Insomnia'],106,150,70,194,120,140,780,130.0,38.453435043785966,6.0,Slow,1250000,1.0,3,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,0.5,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.5,33.0,14.7
150,Mega Mewtwo X,Psychic,Fighting,['Steadfast'],106,190,100,154,100,130,780,130.0,33.04542328371661,6.0,Slow,1250000,1.0,3,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,2.3,127.0,24.0
890,Eternatus,Poison,Dragon,['Pressure'],140,85,95,145,95,130,690,115.0,23.9791576165636,8.0,Slow,1250000,1.0,255,1.0,0.0,0.0,None,1.0,0.5,0.5,0.5,0.25,2.0,0.5,0.5,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,1.0,1.0,20.0,950.0,2.4
493,Arceus,Normal,None,['Multitype'],120,120,120,120,120,120,720,120.0,0.0,4.0,Slow,1250000,1.0,3,1.0,0.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,3.2,320.0,31.3
144,Articuno,Ice,Flying,"['Pressure', 'Snow Cloak']",90,85,100,95,125,85,580,96.66666666666669,13.743685418725534,1.0,Slow,1250000,1.0,3,1.0,0.0,0.0,None,1.0,2.0,1.0,2.0,0.5,1.0,1.0,1.0,0.0,1.0,1.0,0.5,4.0,1.0,1.0,1.0,2.0,1.0,1.7,55.4,19.2
362,Mega Glalie,Ice,None,['Refrigerate'],80,120,80,120,80,100,580,96.66666666666669,17.950549357115012,6.0,Medium Fast,1000000,1.0,75,0.0,1.0,0.0,None,1.0,2.0,1.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.1,350.2,79.4


## Schema Evolution
Iceberg lets you change a table’s logical schema without rewriting files unless you choose to backfill.

In [43]:
%%sql
ALTER TABLE academy.pokemon 
DROP COLUMNS (against_normal, against_fire, against_water, against_electric, against_grass, against_ice, against_fighting, against_poison, against_ground, against_flying, against_psychic, against_bug, against_rock, against_ghost, against_dragon, against_dark, against_steel, against_fairy);

++
||
++
++

### Add a new column to CAST a column and backfill it

In [45]:
%%sql
ALTER TABLE academy.pokemon
ADD COLUMN generation_int INT COMMENT 'generation as integer';

++
||
++
++

In [44]:
%%sql
UPDATE academy.pokemon
SET    generation_int = CAST(generation AS INT)
WHERE  generation_int IS NULL;

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `generation_int` cannot be resolved. Did you mean one of the following? [`generation`, `mean`, `abilities`, `att`, `catch_rate`].; line 2 pos 7;
'UpdateTable [assignment('generation_int, cast(generation#1630 as int))], isnull('generation_int)
+- SubqueryAlias demo.academy.pokemon
   +- RelationV2[number#1616, name#1617, type_1#1618, type_2#1619, abilities#1620, hp#1621, att#1622, def#1623, spa#1624, spd#1625, spe#1626, bst#1627, mean#1628, standard_deviation#1629, generation#1630, experience_type#1631, experience_to_level_100#1632, final_evolution#1633, catch_rate#1634, legendary#1635, mega_evolution#1636, alolan_form#1637, galarian_form#1638, height#1639, ... 2 more fields] demo.academy.pokemon demo.academy.pokemon


In [20]:
%%sql
ALTER TABLE academy.pokemon 
DROP COLUMN generation

++
||
++
++

In [23]:
%%sql
ALTER TABLE academy.pokemon
RENAME COLUMN generation_int TO generation

++
||
++
++

In [20]:
%%sql
ALTER TABLE academy.pokemon 
ADD PARTITION FIELD generation

++
||
++
++

In [23]:
%%sql
CALL demo.system.rewrite_data_files(
  'academy.pokemon'
)

rewritten_data_files_count,added_data_files_count,rewritten_bytes_count,failed_data_files_count
18,134,215603,0


## Iceberg metadata changes

In [24]:
%%sql
SELECT
    COUNT(*)
FROM academy.pokemon.partitions

count(1)
134


In [25]:
%%sql
SELECT *
FROM academy.pokemon.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-07-29 20:04:14.591000,288104134174738007,None,append,s3://warehouse/academy/pokemon/metadata/snap-288104134174738007-1-2397a9a0-35b6-4ed5-9049-d479d2ee05d1.avro,"{'engine-version': '3.5.5', 'added-data-files': '18', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'total-records': '1032', 'spark.app.id': 'local-1753816925020', 'changed-partition-count': '18', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '330021', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '330021', 'total-data-files': '18'}"
2025-07-29 20:04:38.727000,5675170136737628119,288104134174738007,overwrite,s3://warehouse/academy/pokemon/metadata/snap-5675170136737628119-1-0d802715-7de2-4395-b5a9-a0868ca30799.avro,"{'engine-version': '3.5.5', 'added-data-files': '18', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'deleted-data-files': '18', 'deleted-records': '1032', 'total-records': '1032', 'spark.app.id': 'local-1753816925020', 'removed-files-size': '330021', 'changed-partition-count': '18', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '215603', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '215603', 'total-data-files': '18'}"
2025-07-29 20:10:57.420000,6101732590538168055,5675170136737628119,replace,s3://warehouse/academy/pokemon/metadata/snap-6101732590538168055-1-f24a9592-0e98-46c8-a2d3-cf152b88e390.avro,"{'engine-version': '3.5.5', 'added-data-files': '134', 'total-equality-deletes': '0', 'app-id': 'local-1753816925020', 'added-records': '1032', 'deleted-data-files': '18', 'deleted-records': '1032', 'total-records': '1032', 'removed-files-size': '215603', 'changed-partition-count': '152', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '1180084', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '1180084', 'total-data-files': '134'}"


## Row level deletes

In [26]:
%%sql
DELETE FROM academy.pokemon WHERE generation=5

++
||
++
++

In [27]:
%%sql
SELECT COUNT(*) FROM academy.pokemon

count(1)
869


## Time Travel & Rollbacks

Iceberg can query any snapshot by ID or timestamp.

###           Travel using VERSION AS OF (by snapshot_id)

In [28]:
# Grab the earliest snapshot for demonstration
first_snap = spark.sql("SELECT MIN(snapshot_id) AS id FROM academy.pokemon.snapshots").first()["id"]
print(first_snap)

288104134174738007


In [29]:
spark.sql(f"SELECT COUNT(*) FROM academy.pokemon VERSION AS OF {first_snap}").show()

+--------+
|count(1)|
+--------+
|    1032|
+--------+

